In [ ]:
using CSV
using Dates
using DataFramesMeta
using Statistics
using ActuaryUtilities

using PlotlyJS

basepath = "../data"
include("$basepath/datacatalog.jl");

In [2]:
db = CSV.read("$basepath/ESGC Cost Performance Database 2024.csv", DataFrame);
# revenue = CSV.read("storage-revenue-by-tech-duration-node-caiso.csv", DataFrame);
# revenue = CSV.read("storage-revenue-by-tech-duration-node-ercot.csv", DataFrame);
# revenue = CSV.read("storage-revenue-by-tech-duration-node-miso.csv", DataFrame);
revenue = CSV.read("storage-revenue-by-tech-duration-node-lookahead.csv", DataFrame);

In [3]:
@rsubset! db begin
    :year == 2023
    :power_mw == 100
    :estimate_type == "Point"
    :duration in [4, 10, 24, 100]
    :parameter in ["Calendar Life (yrs)", "Total Installed Cost (\$/kW)", "Total Installed Cost (\$/kWh)"]
end;

In [4]:
@select!(db, :technology, :duration, :parameter, :value);

In [5]:
db = unstack(db, [:technology, :duration], :parameter, :value);

In [6]:
@rename! db begin 
    :kW_cost = :"Total Installed Cost (\$/kW)"
    :kWh_cost = :"Total Installed Cost (\$/kWh)"
    :lifespan = :"Calendar Life (yrs)"
end;

In [7]:
leftjoin!(revenue, db, on = [:duration, :technology]);

In [8]:
@rtransform!(revenue, :total_cost = :kWh_cost * :duration * 1000);

In [9]:
# PRA revenue = $20/MW-day
@rtransform!(revenue, :pra_revenue = 7300 * 1);
@rtransform!(revenue, :simple_payback = :total_cost / (:revenue + :pra_revenue));
@rtransform!(revenue, :simple_payback_itc = (:total_cost * 2/3) / (:revenue + :pra_revenue));

In [10]:
# wacc = 0.07

In [11]:
# @chain revenue begin
#     # @rsubset occursin.("Li", :technology)
#     # @rsubset :duration_hr == 4
#     @orderby :simple_payback
#     # @rsubset :Row < 10
# end

In [20]:
@chain revenue begin
    @rsubset :technology in ["Iron Air"]
    @rsubset :duration == 100 # || :duration == 4
    @rsubset :location == "nobles"
    @orderby :technology
end

Row,duration,location,lookahead,technology,revenue,lifespan,kW_cost,kWh_cost,total_cost,pra_revenue,simple_payback,simple_payback_itc
,Int64,String7,Int64,String31,Float64,Float64?,Float64?,Float64?,Float64?,Int64,Float64?,Float64?
1,100,nobles,24,Iron Air,92193.9,25.0,missing,35.0,3.5e6,7300,35.178,23.452
2,100,nobles,48,Iron Air,92987.0,25.0,missing,35.0,3.5e6,7300,34.8999,23.2666
3,100,nobles,72,Iron Air,93207.0,25.0,missing,35.0,3.5e6,7300,34.8234,23.2156
4,100,nobles,96,Iron Air,93470.9,25.0,missing,35.0,3.5e6,7300,34.7323,23.1548
5,100,nobles,168,Iron Air,93721.1,25.0,missing,35.0,3.5e6,7300,34.6462,23.0975
6,100,nobles,240,Iron Air,94115.8,25.0,missing,35.0,3.5e6,7300,34.5114,23.0076
